In [0]:
%run ./api_connector

In [0]:
import uuid, json

class DatabricksApi(ApiConnector):
    def __init__(self, scope, base_url, secret_key, timeout=30):
        super().__init__(scope, base_url)
        self.token = self.get_auth(scope, secret_key)
        self.headers = {
            'Authorization': f'Bearer {self.token}'
        }

    
    def get_auth(self, scope, secret_key):
        return dbutils.secrets.get(scope=scope, key=secret_key)
    

    def run_job(self, payload: dict) -> dict:
        endpoint = "/api/2.2/jobs/run-now"

        if 'idempotency_token' not in payload:
            idempotency_token = str(uuid.uuid4())
            payload["idempotency_token"] = idempotency_token
        else:
            idempotency_token = payload["idempotency_token"]
        
        response = self.request("POST", endpoint, data=json.dumps(payload))
        
        data = response.json()
        data['idempotency_token'] = idempotency_token

        return data